In [0]:
"""
Descripcion:
    Lee la información directamente de OCI, realiza las transformaciones necesarias y carga la información directo en las tablas de OCI
    483 - Interpretación de Contenido
Trámite:
    354 - Desmarca de crédito de vivienda por 43 BIS
Tablas input:
    CIERREN_ETL.TTSISGRAL_ETL_LEE_ARCHIVO
Tablas output:
    CIERREN_ETL.TTAFOTRAS_ETL_MARCA_DESMARCA
Tablas Delta:
    DELTA_INCO_01_{params.sr_folio}
    DELTA_INCO_02_{params.sr_folio}
Archivos SQL:
    INCO_MAR_CUE_EXT_0100_OCI_LEE_ARCHIVO
    INCO_MAR_CUE_TRN_0200_DBK_TRANSF_INFONAVIT
"""

In [0]:
%run "./startup"

In [0]:
params = WidgetParams({
    "sr_folio": str,
    "sr_proceso": str,
    "sr_subproceso": str,
    "sr_subetapa": str,

})
params.validate()

In [0]:
# Cargar configuración de entorno
conf = ConfManager()

# Inicializar managers
query = QueryManager()
db = DBXConnectionManager()

In [0]:
statement_trans_info = query.get_statement(
    "NB_PATRIF_MCO_0100_COMUNICACION_01_001_OCI_TRANS_INFO.sql",
    SR_FOLIO=params.sr_folio,
    SR_PROCESO=params.sr_proceso,
    SR_SUBPROCESO=params.sr_subproceso
)
trans_info_df = db.read_data("default", statement_trans_info)
db.write_delta(f"TEMP_TRANS_INFO_{params.sr_folio}", trans_info_df, "overwrite")


if conf.debug:
    #display(db.read_delta(f"DELTA_INCO_01_{params.sr_folio}").count())
    display(db.read_delta(f"TEMP_TRANS_INFO_{params.sr_folio}"))


In [0]:
statement_suf_saldos = query.get_statement(
    "NB_PATRIF_MCO_0100_COMUNICACION_01_002_OCI_SUF_SALDOS.sql",
    SR_FOLIO=params.sr_folio,
    SR_SUBPROCESO=params.sr_subproceso
)
suf_saldos_df = db.read_data("default", statement_suf_saldos)
db.write_delta(f"TEMP_SUF_SALDOS_{params.sr_folio}", suf_saldos_df, "overwrite")


if conf.debug:
    #display(db.read_delta(f"DELTA_INCO_01_{params.sr_folio}").count())
    display(db.read_delta(f"TEMP_SUF_SALDOS_{params.sr_folio}"))

In [0]:
statement_join = query.get_statement(
    "NB_PATRIF_MCO_0100_COMUNICACION_01_003_DELTA_JOIN.sql",
    CATALOG_SCHEMA=f"{SETTINGS.GENERAL.CATALOG}.{SETTINGS.GENERAL.SCHEMA}",
    TRANS_INFO_TABLE=f"TEMP_TRANS_INFO_{params.sr_folio}",
    SUF_SALDOS_TABLE=f"TEMP_SUF_SALDOS_{params.sr_folio}"
)
join_result_df = db.sql_delta(statement_join)
db.write_delta(f"TEMP_JOIN_RESULT_{params.sr_folio}", join_result_df, "overwrite")



if conf.debug:
    #display(db.read_delta(f"DELTA_INCO_01_{params.sr_folio}").count())
    display(db.read_delta(f"TEMP_JOIN_RESULT_{params.sr_folio}"))

In [0]:
# Limpiar datos del notebook y liberar memoria
CleanUpManager.cleanup_notebook(locals())
